### Chapter 4 - Training Models
---

* **1. What Linear Regression training algorithm can you use if you have a training set with millions of features?**
> Gradient Descent training algorithms (e.g. Batch, Stochastic, Mini-Batch) are all well suited to training a Linear Regression model with a large number of features, whereas using the Normal Equation doesn't scale well and is best suited to data with fewer features. 

* **2. Suppose the features in your training set have very different scales. What algorithms might suffer from this, and how? What can you do about it?**
> Gradient Descent algorithms require scaled data as they perform very poorly otherwise, either taking an extremely long time to reach the optimal model parameters or not doing so at all. This eventuality can be avoided by scaling the features in the data during preprocessing (e.g. by using a scaling function from scikit-learn) so they all fall within a similar range of values.

* **3. Can Gradient Descent get stuck in a local minimum when training a Logistic Regression model?**
> No, as the cost function for Logistic Regression (the log-loss function) is convex and therefore only has a global minimum, which a Gradient Descent algorithm will always find given enough training time and learning rate that's not too large.

* **4. Do all Gradient Descent algorithms lead to the same model provided you let them run long enough?**
> No, they do not. Batch Gradient Descent will descend smoothly towards the optimal parameters due to the fact it samples the entire training set at every iteration, whereas Stochastic GD and Mini-Batch sample a single random instance and a random batch of instances respectively, and due to this randomness, these algorithms don't settle at the global minimum like Batch GD does, they 'jump' around the minimum, never truly converging.

* **5. Suppose you use Batch Gradient Descent and you plot the validation error at every epoch. If you notice that the validation error consistently goes up, what is likely going on? How can you fix this?**
> If the training error is not increasing, then overfitting is likely the problem which can be possibly solved by decreasing model complexity or feeding more training data into the model. Alternatively, if the training error is also increasing then it's more likely the case that the learning rate is too high and the model is not able to converge on the optimal solution, reducing the learning rate to decrease the GD step size will likely help in this situation.

* **6. Is it a good idea to stop Mini-batch Gradient Descent immediately when the validation error goes up?**
> No, it is not. This is because Mini-batch GD uses random samples on each iteration, so unlike Batch gradient descent, an increased validation error doesn't necessarily mean that the algorithm has reached some form of minimum. Mini-batch GD will 'jump' around due to randomness involved in the process, so it's best to set the algorithm to stop when the validation error hasn't decreased in a certain number of epochs (10, for example), at which point the model should revert to the parameters which returned the lowest error.

* **7. Which Gradient Descent algorithm (among those we discussed) will reach the vicinity of the optimal solution the fastest? Which will actually converge? How can you make the others converge as well?**
> Stochastic GD is likely to reach the vicinity of the optimal solution fastest as it only uses one training instance per iteration, making it a much quicker option than either of the Batch GD algorithms. However, only Batch GD will actually converge fully as it uses the entire training set at each iteration. Stochastic and Mini-batch GD can be coerced into converging via stimulated annealing, which is the process of gradually reducing the learning rate during the training process, the idea being that the algorithm will take smaller and smaller steps until it converges on the global minimum.

* **8. Suppose you are using Polynomial Regression. You plot the learning curves and you notice that there is a large gap between the training error and the validation error. What is happening? What are three ways to solve this?**
> If the validation error is much higher than the training error, the model is likely overfitting the training set. If possible, exposing the model to more data by increasing the size of the training set will likely help the situation. Reducing model complexity by either selecting a simpler algorithm with fewer polynomial degrees or introducing regularization (Ridge or Lasso for example) to constrain the model parameters should also combat this issue.

* **9. Suppose you are using Ridge Regression and you notice that the training error and the validation error are almost equal and fairly high. Would you say that the model suffers from high bias or high variance? Should you increase the regularization hyperparameter α or reduce it?**
> Both errors being similarly high likely means the model is underfitting the data; this corresponds to high bias, as the model is not complex enough to give satisfactory predictions. The regularization hyperparameter should be decreased in this instance to lessen the constraints on the model and allow a better fit to the training set.

* **10. Why would you want to use:** 
  * **Ridge Regression instead of Linear Regression?**
  > Ridge is a good default algorithm to use instead of simple Linear Regression as introducing regularization, even if it's to a small degree, is generally always a good idea as it constrains the model from becoming too complex and overfitting the training data.
  * **Lasso instead of Ridge Regression?**
  > Lasso automatically discards unimportant features, so would be the perfect choice if you had data with a large number of features, many of which you believe to be superfluous, as Lasso will just ignore them and consider the important predictors.
  * **Elastic Net instead of Lasso?**
  > Elastic Net is a weighted combination of Lasso and Ridge (governed by a fraction, `r`) and is typically preferred to Lasso as Lasso can behave unpredictably when there are more features than training instances or when strong correlation exists between features.

* **11. Suppose you want to classify pictures as outdoor/indoor and daytime/nighttime. Should you implement two Logistic Regression classifiers or one Softmax Regression classifier?**
> This is a multioutput problem, as there are two different classes for each of the two outputs, therefore this is suited to using two Logistic Regression classifiers, not a multinomial/Softmax Regression classifier which is used for multiclass problems.

* **12. Implement Batch Gradient Descent with early stopping for Softmax Regression (without using Scikit-Learn).**
> Batch Gradient Descent means that we'll be using the entire training set at every iteration, and early stopping means that once the validation error increases instead of decreasing, we'll stop the training process. Softmax regression is effectively just multiclass Logistic Regression.

In [0]:
import numpy as np
np.random.seed(42)

iris = datasets.load_iris()                                                      # loading in all 150 training instances; there are two features and three classes of flower
X = iris['data'][:,(2,3)]
y = iris['target']

X_biased = np.c_[np.ones([len(X), 1]), X]                                        # adding bias term of 1 to every instance, as required for logistic regression

print('Shape of data:', X_biased.shape)
print('Shape of target:', y.shape)

Shape of data: (150, 3)
Shape of target: (150,)


Now, we need to split the data into a training and test set without using scikit-learn. This is done by shuffling the indices of the data, selecting the training set size (80%) and then indexing the data using the shuffled indices, thereby randomly selecting a test set from the data.

In [0]:
indices_shuffled = np.random.permutation(len(y))
train_size = int(len(y)*0.8)
train_indices = indices_shuffled[:train_size]
test_indices = indices_shuffled[train_size:]
X_train, X_test = X_biased[train_indices], X_biased[test_indices]
y_train, y_test = y[train_indices], y[test_indices]
print('Shape of training set:', X_train.shape)
print('Shape of test set:', X_test.shape)

Shape of training set: (120, 3)
Shape of test set: (30, 3)


The next step is one-hot encoding the target variables, as currently due to the fact there's three classes (0, 1 and 2), the target values are not binary. This is done by....

In [0]:
print(y_test)

def onehotencoder(target):
    encoded_target = np.zeros((len(target),3), dtype=np.int64)
    for i in range(len(target)):
        if target[i] == 0:
            encoded_target[i, 0] = 1
        elif target[i] == 1:
            encoded_target[i, 1] = 1
        elif target[i] == 2:
            encoded_target[i, 2] = 1
    return encoded_target

y_test_enc = onehotencoder(y_test)
print(y_test_enc)

[0 2 0 2 0 0 1 0 2 1 0 2 2 1 1 2 2 0 2 1 1 2 0 1 0 2 0 1 2 1]
[[1 0 0]
 [0 0 1]
 [1 0 0]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [1 0 0]
 [0 0 1]
 [0 1 0]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [0 1 0]
 [0 1 0]
 [0 0 1]
 [0 0 1]
 [1 0 0]
 [0 0 1]
 [0 1 0]
 [0 1 0]
 [0 0 1]
 [1 0 0]
 [0 1 0]
 [1 0 0]
 [0 0 1]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [0 1 0]]
